## Lab 1.2 ENHANCED: File-Based SQLite Persistence with Pause & Resume

**What's Different from Original Lab 1.2:**
- ❌ Original: Uses `:memory:` - lost when program ends
- ✅ Enhanced: Uses actual file - survives restarts!

**Real-World Scenario:**
An HR assistant helping with employee onboarding across multiple days:
- **Day 1:** Employee starts onboarding, completes documents
- **Night:** System shuts down
- **Day 2:** Employee returns, conversation continues from where they left off

**You'll Learn:**
1. Create SQLite database in a file
2. Save conversation state
3. Close/pause the application
4. Resume from saved state
5. Verify data persistence across sessions

### Step 1: Setup and Database Creation

In [ ]:
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from typing import Annotated
import sqlite3
import os

# Database file configuration
DB_FILE = "hr_assistant_persistent.db"

# Remove old database for fresh start (optional)
if os.path.exists(DB_FILE):
    os.remove(DB_FILE)
    print(f"🗑️  Removed existing database: {DB_FILE}")

print(f"\n📁 Database file: {os.path.abspath(DB_FILE)}")
print("✅ Ready to create persistent SQLite database!")

### Step 2: Define HR Onboarding Tools

In [ ]:
# Enhanced HR tools for onboarding workflow

@tool
def verify_documents(employee_id: Annotated[str, "Employee ID"]) -> str:
    """Verify that employee has submitted all required documents."""
    # Simulated document check
    documents = ["ID Proof", "Address Proof", "Educational Certificates", "Previous Employment Letter"]
    return f"✅ All documents verified for employee {employee_id}: {', '.join(documents)}"

@tool
def assign_equipment(employee_id: Annotated[str, "Employee ID"], 
                     department: Annotated[str, "Department name"]) -> str:
    """Assign laptop and equipment to employee based on department."""
    equipment = {
        "Engineering": "MacBook Pro 16\", Monitor, Keyboard, Mouse",
        "Marketing": "Dell Laptop, Monitor, Marketing Kit",
        "Sales": "Dell Laptop, Mobile Phone, Sales Materials",
        "HR": "Dell Laptop, Monitor, HR Software Access"
    }
    assigned = equipment.get(department, "Standard Laptop Package")
    return f"✅ Equipment assigned to {employee_id}: {assigned}"

@tool
def schedule_orientation(employee_id: Annotated[str, "Employee ID"],
                        employee_name: Annotated[str, "Employee name"]) -> str:
    """Schedule orientation session for new employee."""
    return f"✅ Orientation scheduled for {employee_name} ({employee_id}):\n" \
           f"   📅 Date: Monday, 9:00 AM\n" \
           f"   📍 Location: Conference Room A\n" \
           f"   ⏱️  Duration: 3 hours"

@tool
def setup_accounts(employee_id: Annotated[str, "Employee ID"],
                  email: Annotated[str, "Employee email"]) -> str:
    """Setup email and system accounts for employee."""
    return f"✅ Accounts created for {employee_id}:\n" \
           f"   📧 Email: {email}\n" \
           f"   🔐 VPN Access: Enabled\n" \
           f"   💻 System Login: Created\n" \
           f"   📱 Slack: Account created"

# Collect all tools
onboarding_tools = [
    verify_documents,
    assign_equipment,
    schedule_orientation,
    setup_accounts
]

print("✅ HR Onboarding tools defined:")
for tool in onboarding_tools:
    print(f"   - {tool.name}: {tool.description}")

### Step 3: SESSION 1 - Initial Onboarding Conversation

**Scenario:** Employee starts onboarding on Day 1

In [ ]:
print("="*80)
print("🚀 SESSION 1: Starting Employee Onboarding (Day 1)")
print("="*80)

# Create SQLite connection to FILE (not :memory:)
connection = sqlite3.connect(DB_FILE, check_same_thread=False)
checkpointer = SqliteSaver(connection)

print(f"\n✅ Connected to database: {DB_FILE}")
print(f"📊 Database location: {os.path.abspath(DB_FILE)}")

# Create agent with persistent checkpointing
persistent_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=onboarding_tools,
    checkpointer=checkpointer,
    prompt="""You are a helpful HR Onboarding Assistant.
    
    Your responsibilities:
    - Guide new employees through the onboarding process
    - Verify documents
    - Assign equipment
    - Schedule orientation
    - Setup accounts
    
    Be friendly, professional, and remember all conversation context.
    Guide employees step-by-step through onboarding."""
)

# Thread ID for this employee's onboarding
config = {"configurable": {"thread_id": "employee_priya_onboarding_2024"}}

print(f"\n🆔 Thread ID: employee_priya_onboarding_2024")
print("\n" + "-"*80)

In [ ]:
# Turn 1: Employee introduces themselves
print("\n👤 TURN 1: Employee Introduction")
print("-"*80)

result = persistent_agent.invoke(
    {"messages": [{
        "role": "user",
        "content": "Hi! I'm Priya Sharma, employee ID EMP-2024-101. I'm joining the Engineering department today and need to complete my onboarding."
    }]},
    config
)

print(f"\n🤖 Assistant: {result['messages'][-1].content}")
print(f"\n💾 State saved to: {DB_FILE}")

In [ ]:
# Turn 2: Document verification
print("\n📄 TURN 2: Document Verification")
print("-"*80)

result = persistent_agent.invoke(
    {"messages": [{
        "role": "user",
        "content": "Yes, I've uploaded all my documents. Can you verify them?"
    }]},
    config
)

print(f"\n🤖 Assistant: {result['messages'][-1].content}")
print(f"\n💾 State updated in: {DB_FILE}")

In [ ]:
# Turn 3: Equipment request
print("\n💻 TURN 3: Equipment Assignment")
print("-"*80)

result = persistent_agent.invoke(
    {"messages": [{
        "role": "user",
        "content": "Great! Now I need my laptop and equipment for the Engineering department."
    }]},
    config
)

print(f"\n🤖 Assistant: {result['messages'][-1].content}")
print(f"\n💾 State saved to: {DB_FILE}")

### Step 4: Check State and Close Connection (PAUSE)

**Simulating:** End of Day 1 - Employee goes home, system shuts down

In [ ]:
print("\n" + "="*80)
print("📊 CHECKING STATE BEFORE PAUSE")
print("="*80)

# Get current state
current_state = persistent_agent.get_state(config)

print(f"\n✅ Current state saved successfully!")
print(f"\n📝 State Information:")
print(f"   - Total messages in conversation: {len(current_state.values['messages'])}")
print(f"   - Thread ID: {config['configurable']['thread_id']}")
print(f"   - Checkpoint ID: {current_state.config['configurable'].get('checkpoint_id', 'N/A')[:16]}...")

# Show conversation summary
print(f"\n💬 Conversation Summary:")
for i, msg in enumerate(current_state.values['messages'][-6:], 1):
    role = msg.__class__.__name__
    content = msg.content[:100] + "..." if len(msg.content) > 100 else msg.content
    print(f"   {i}. {role}: {content}")

In [ ]:
print("\n" + "="*80)
print("⏸️  PAUSING SESSION - Simulating End of Day / Application Shutdown")
print("="*80)

# Verify database file exists
print(f"\n✅ Database file exists: {os.path.exists(DB_FILE)}")
print(f"📁 File location: {os.path.abspath(DB_FILE)}")
print(f"💾 File size: {os.path.getsize(DB_FILE)} bytes")

# Close connection (simulate application shutdown)
connection.close()
print(f"\n🔌 Database connection closed")
print(f"\n⏸️  SESSION 1 PAUSED")
print(f"\n💡 The conversation state is now saved to disk in: {DB_FILE}")
print(f"💡 We can now restart the application and resume the conversation!")

# Clean up variables to simulate fresh start
del persistent_agent
del checkpointer
print(f"\n🧹 Cleaned up agent and checkpointer from memory")
print(f"\n" + "="*80)

### Step 5: Verify Database Contents (Optional Inspection)

**Let's peek inside the SQLite database to see what's stored!**

In [ ]:
print("\n" + "="*80)
print("🔍 INSPECTING DATABASE CONTENTS")
print("="*80)

# Open database for inspection only (read-only)
inspect_conn = sqlite3.connect(DB_FILE)
cursor = inspect_conn.cursor()

# List all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print(f"\n📋 Tables in database:")
for table in tables:
    print(f"   - {table[0]}")

# Count checkpoints
cursor.execute("SELECT COUNT(*) FROM checkpoints;")
checkpoint_count = cursor.fetchone()[0]
print(f"\n💾 Total checkpoints stored: {checkpoint_count}")

# Show checkpoint details
cursor.execute("""
    SELECT thread_id, checkpoint_id, parent_checkpoint_id
    FROM checkpoints
    ORDER BY checkpoint_id
    LIMIT 5;
""")
checkpoints = cursor.fetchall()

print(f"\n📝 Recent Checkpoints:")
for i, (thread_id, checkpoint_id, parent_id) in enumerate(checkpoints, 1):
    print(f"   {i}. Thread: {thread_id}")
    print(f"      Checkpoint: {checkpoint_id[:24]}...")
    print(f"      Parent: {parent_id[:24] if parent_id else 'None'}...\n")

# Count writes
cursor.execute("SELECT COUNT(*) FROM writes;")
writes_count = cursor.fetchone()[0]
print(f"💾 Total writes stored: {writes_count}")

cursor.close()
inspect_conn.close()

print(f"\n✅ Database inspection complete")
print(f"\n" + "="*80)

### Step 6: SESSION 2 - Resume After Restart (RESUME)

**Scenario:** Next day (Day 2) - Employee returns, application restarts, conversation resumes

In [ ]:
print("\n" + "="*80)
print("🔄 SESSION 2: Resuming Employee Onboarding (Day 2)")
print("="*80)
print("\n⏰ Simulating: Next day, application restarted, employee returns...")

# Reconnect to the SAME database file
new_connection = sqlite3.connect(DB_FILE, check_same_thread=False)
new_checkpointer = SqliteSaver(new_connection)

print(f"\n✅ Reconnected to database: {DB_FILE}")
print(f"✅ Loaded existing checkpointer")

# Create NEW agent instance (simulating fresh application start)
resumed_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=onboarding_tools,
    checkpointer=new_checkpointer,  # Uses the SAME database!
    prompt="""You are a helpful HR Onboarding Assistant.
    
    Your responsibilities:
    - Guide new employees through the onboarding process
    - Verify documents
    - Assign equipment
    - Schedule orientation
    - Setup accounts
    
    Be friendly, professional, and remember all conversation context.
    Guide employees step-by-step through onboarding."""
)

print(f"\n✅ Created new agent instance (fresh start)")
print(f"\n🔗 Using SAME thread ID to resume conversation")
print(f"\n" + "-"*80)

In [ ]:
# Retrieve previous state
print("\n📥 RETRIEVING PREVIOUS STATE")
print("-"*80)

# Use the SAME config (thread_id) to retrieve state
config = {"configurable": {"thread_id": "employee_priya_onboarding_2024"}}

retrieved_state = resumed_agent.get_state(config)

print(f"\n✅ Successfully retrieved previous state!")
print(f"\n📊 Retrieved State Information:")
print(f"   - Messages in history: {len(retrieved_state.values['messages'])}")
print(f"   - Thread ID: {config['configurable']['thread_id']}")
print(f"   - Checkpoint ID: {retrieved_state.config['configurable'].get('checkpoint_id', 'N/A')[:16]}...")

print(f"\n💬 Previous Conversation Recap:")
print(f"-"*80)
for i, msg in enumerate(retrieved_state.values['messages'][-6:], 1):
    role = msg.__class__.__name__
    content = msg.content[:150] + "..." if len(msg.content) > 150 else msg.content
    print(f"\n{i}. {role}:")
    print(f"   {content}")

print(f"\n" + "-"*80)
print(f"\n✅ The agent remembers EVERYTHING from yesterday!")

In [ ]:
# Turn 4: Continue conversation - Schedule orientation
print("\n📅 TURN 4: Continuing Onboarding - Orientation Scheduling")
print("-"*80)

result = resumed_agent.invoke(
    {"messages": [{
        "role": "user",
        "content": "Thanks! I received my equipment. Can you schedule my orientation session?"
    }]},
    config  # Same thread_id - continues from where we left off!
)

print(f"\n🤖 Assistant: {result['messages'][-1].content}")
print(f"\n✅ Agent continued seamlessly from previous session!")

In [ ]:
# Turn 5: Setup accounts
print("\n🔐 TURN 5: Account Setup")
print("-"*80)

result = resumed_agent.invoke(
    {"messages": [{
        "role": "user",
        "content": "Perfect! Now I need my email and system accounts. My email should be priya.sharma@company.com"
    }]},
    config
)

print(f"\n🤖 Assistant: {result['messages'][-1].content}")

In [ ]:
# Turn 6: Test memory - Ask about earlier conversation
print("\n🧠 TURN 6: Testing Memory - Recall from Day 1")
print("-"*80)

result = resumed_agent.invoke(
    {"messages": [{
        "role": "user",
        "content": "Can you remind me what we did yesterday and what my employee ID is?"
    }]},
    config
)

print(f"\n🤖 Assistant: {result['messages'][-1].content}")
print(f"\n✅ Agent remembered details from PREVIOUS SESSION (Day 1)!")

### Step 7: Final State Check and Cleanup

In [ ]:
print("\n" + "="*80)
print("📊 FINAL STATE CHECK")
print("="*80)

final_state = resumed_agent.get_state(config)

print(f"\n✅ Onboarding Session Complete!")
print(f"\n📝 Final Statistics:")
print(f"   - Total messages exchanged: {len(final_state.values['messages'])}")
print(f"   - Messages from Day 1: 6 (3 user + 3 assistant)")
print(f"   - Messages from Day 2: {len(final_state.values['messages']) - 6}")
print(f"   - Thread ID: {config['configurable']['thread_id']}")

# Get state history to show all checkpoints
state_history = list(resumed_agent.get_state_history(config))
print(f"\n💾 Total checkpoints created: {len(state_history)}")
print(f"\n📈 Checkpoint progression:")
for i, state in enumerate(state_history[:5], 1):
    msg_count = len(state.values.get('messages', []))
    print(f"   Checkpoint {i}: {msg_count} messages")

if len(state_history) > 5:
    print(f"   ... and {len(state_history) - 5} more checkpoints")

In [ ]:
# Close connection
new_connection.close()
print(f"\n🔌 Database connection closed")
print(f"\n💾 All data safely persisted in: {os.path.abspath(DB_FILE)}")
print(f"\n" + "="*80)

### Step 8: Verification - Prove Persistence Works!

In [ ]:
print("\n" + "="*80)
print("🔬 VERIFICATION: Demonstrating True Persistence")
print("="*80)

print("\n1️⃣ Database file still exists after closing:")
print(f"   ✅ File exists: {os.path.exists(DB_FILE)}")
print(f"   📁 Location: {os.path.abspath(DB_FILE)}")
print(f"   💾 Size: {os.path.getsize(DB_FILE):,} bytes")

print("\n2️⃣ Can read data without agent:")
verify_conn = sqlite3.connect(DB_FILE)
cursor = verify_conn.cursor()
cursor.execute("SELECT COUNT(*) FROM checkpoints WHERE thread_id = ?", 
               (config['configurable']['thread_id'],))
thread_checkpoints = cursor.fetchone()[0]
print(f"   ✅ Found {thread_checkpoints} checkpoints for thread: {config['configurable']['thread_id'][:30]}...")
cursor.close()
verify_conn.close()

print("\n3️⃣ Data survives Python kernel restart:")
print(f"   ✅ You can restart this notebook and reconnect to: {DB_FILE}")
print(f"   ✅ All conversation history will be preserved")
print(f"   ✅ Can resume from ANY checkpoint in the history")

print("\n" + "="*80)
print("✅ VERIFICATION COMPLETE - Persistence is working!")
print("="*80)

## 🎯 Summary: What We Learned

### Key Differences: Memory vs File-Based Persistence

| Feature | `:memory:` (Original Lab 1.2) | File-Based (Enhanced Lab 1.2) |
|---------|------------------------------|-------------------------------|
| **Storage** | RAM only | Disk file |
| **Survives restart** | ❌ No | ✅ Yes |
| **Use case** | Testing, demos | Production, real apps |
| **Speed** | Faster | Slightly slower |
| **Persistence** | Lost on exit | Permanent |

### What We Demonstrated

✅ **Session 1 (Day 1):**
- Created file-based SQLite database
- Started employee onboarding conversation
- Verified documents
- Assigned equipment
- Saved state to disk
- Closed application

✅ **Session 2 (Day 2):**
- Reconnected to same database file
- Retrieved complete conversation history
- Continued onboarding seamlessly
- Agent remembered all previous context
- Completed remaining steps

### Production Benefits

🚀 **Real-World Applications:**
- Multi-day onboarding processes
- Long-running support conversations
- Human-in-the-loop approvals
- Audit trails and compliance
- Resume after system failures

### Next Steps

To use in production:
1. Replace `sqlite3.connect(DB_FILE)` with proper connection management
2. Add error handling for database operations
3. Implement backup strategy for database file
4. Consider PostgreSQL for multi-user scenarios
5. Add authentication/authorization for thread access

## 💡 Quick Reference: File-Based SQLite Pattern

```python
# Setup
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver

# Create/open database FILE
DB_FILE = "my_agent.db"
conn = sqlite3.connect(DB_FILE, check_same_thread=False)
checkpointer = SqliteSaver(conn)

# Create agent
agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[...],
    checkpointer=checkpointer
)

# Use with thread_id
config = {"configurable": {"thread_id": "unique_thread_id"}}
agent.invoke({"messages": [...]}, config)

# Close when done
conn.close()

# Later: Resume
new_conn = sqlite3.connect(DB_FILE, check_same_thread=False)
new_checkpointer = SqliteSaver(new_conn)
new_agent = create_agent(..., checkpointer=new_checkpointer)
new_agent.invoke({"messages": [...]}, config)  # Same thread_id!
```